# GRASS Tool with Parameters

## Interface Definition for Scripts

Since GRASS tools are executable scripts (or generally programs), the interface of a GRASS tool is the command line interface of a Python script. A dedicated function _grass.script.parser_ takes care of processing the command line arguments based on the interface description specified in a Python comment with a key-value syntax defined by GRASS GIS.

The following is an example of a script which takes two parameters: name of a vector map and name of a raster map:

In [26]:
%%writefile viewshed.py
#!/usr/bin/env python

# %module
# % description: Compute viewshed and compute statistics about visible parts of sample layers
# % keyword: viewshed
# % keyword: geometry
# %end
# %option G_OPT_R_ELEV
# % description: Name of input elevation raster map
# %end
# %option G_OPT_M_COORDS
# %end
# %option G_OPT_R_INPUTS
# %end
# %option G_OPT_F_OUTPUT
# % guisection: Output
# %end
# %option
# % key: format
# % type: string
# % required: yes
# % options: json,csv
# % label: Output format
# % descriptions: json;JSON (JavaScript Object Notation);csv;CSV (Comma Separated Values)
# % answer: json
# % guisection: Output
# %end


import atexit
import subprocess
import sys
import csv
import json
import io

import grass.script as gs


def output_results(results, sample_rasters, file_name, file_format):
    if file_format == "json":
        with open(file_name, "w", encoding="utf-8") as json_file:
            json.dump(results, json_file, ensure_ascii=False, indent=4)
    elif file_format == "csv":
        with open(file_name, "w", newline="", encoding="utf-8") as csv_file:
            header = ["name"]
            header.extend(results[sample_rasters[0]].keys())
            writer = csv.DictWriter(csv_file, fieldnames=header)
            writer.writeheader()
            for key, value in results.items():
                row = {"name": key}
                row.update(value)
                writer.writerow(row)
    else:
        raise ValueError(f"Unsupported or invalid format: {file_format}")


def clean(name):
    gs.run_command("g.remove", type="raster", name=name, flags="f", superquiet=True)


def viewshed(
    elevation,
    coordinates,
    sample_rasters,
    output,
    file_format,
):
    viewshed = gs.append_node_pid("tmp_viewshed")
    atexit.register(clean, viewshed)
    gs.run_command(
        "r.viewshed",
        input=elevation,
        output=viewshed,
        coordinates=coordinates,
        flags="cb",
    )
    gs.run_command("r.null", map=viewshed, setnull=0)
    results = {}
    for name in sample_rasters:
        table_data = gs.read_command(
            "r.univar",
            map=name,
            zones=viewshed,
            quiet=True,
            flags="t",
            separator="comma",
        )
        # While we could use .strip().splitlines()[-1].split(",") here,
        # using a proper CSV reader is more robust.
        reader = csv.DictReader(io.StringIO(table_data))
        for row in reader:
            del row["zone"]
            del row["label"]
            del row["non_null_cells"]
            del row["null_cells"]
            results[name] = row
    output_results(
        results=results,
        sample_rasters=sample_rasters,
        file_name=output,
        file_format=file_format,
    )


def main():
    options, flags = gs.parser()
    coordinates = options["coordinates"].split(",")
    sample_rasters = options["input"].split(",")
    viewshed(
        elevation=options["elevation"],
        coordinates=(float(coordinates[0]), float(coordinates[1])),
        sample_rasters=sample_rasters,
        output=options["output"],
        file_format=options["format"],
    )


if __name__ == "__main__":
    main()

Overwriting vector_to_raster.py


As before, we will make the script executable:

In [5]:
!chmod u+x vector_to_raster.py

Running the script with `--help` gives its interface described for command line use:

In [27]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./vector_to_raster.py elevation=elevation coordinates=637_657,222_899 input=elevation,ndvi output="data.txt" format="csv" --o
!cat data.txt

Starting GRASS GIS...
Cleaning up temporary files...
Executing <./vector_to_raster.py elevation=elevation coordinates=637_657,222_899 input=elevation,ndvi output=data.txt format=csv --o> ...
Computing events...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Computing visibility...
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
Writing output raster map...
   0   6  12  18  24  30  36  42  48  54  60  66  72  78  84  90  96 100
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
Removing raster <tmp_viewshed_lemur_3359509>
Execution of <./vector_to_raster.py elevation=elevation coordinates

In [3]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec i.vi red=lsat7_2002_30 nir=lsat7_2002_40 viname=ndvi output=ndvi

Starting GRASS GIS...
Cleaning up temporary files...
Executing <i.vi red=lsat7_2002_30 nir=lsat7_2002_40 viname=ndvi output=ndvi> ...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Execution of <i.vi red=lsat7_2002_30 nir=lsat7_2002_40 viname=ndvi output=ndvi> finished.
Cleaning up temporary files...


Running the script with `--html-description` gives the command line interface described in HTML which later becomes a part of the tool's HTML documentation:

In [51]:
!grass ~/grassdata/nc_spm_grass7/foss4g --exec r.univar map=elevation zones=viewshed -t separator=comma

Starting GRASS GIS...
Cleaning up temporary files...
Executing <r.univar map=elevation zones=viewshed -t separator=comma> ...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
zone,label,non_null_cells,null_cells,min,max,range,mean,mean_of_abs,stddev,variance,coeff_var,sum,sum_abs
1,,121,0,65.0054626464844,152.262680053711,87.2572174072266,115.819097786895,115.819097786895,20.486681456427,419.704117097109,17.6885175656626,14014.1108322144,14014.1108322144
Execution of <r.univar map=elevation zones=viewshed -t separator=comma> finished.
Cleaning up temporary files...


# Outputs

## CSV

```
csv.DictWriter()
```

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec r.univar map=elevation zones=viewshed -t separator=comma format=json